In [2]:
import pandas as pd
import polars as pl

# Read the Excel file into a pandas DataFrame
excel_path = r'C:\Users\Desktop\Name_replacement.xlsx'
df_mappings = pd.read_excel(excel_path, engine='openpyxl')

# Your original Polars DataFrame
df = pl.DataFrame({
    "Name": ["Alice", "Bob", "Charlie", "Bob"],
    "City": ["New York", "Los Angeles", "Chicago", "New York"],
    "Age": [25, 30, 35, 40]
})

# Convert the original Polars DataFrame to pandas for easier row-wise operations
df_pd = df.to_pandas()

# Initialize the 'Changed' column to 'N' for all rows indicating no changes initially
df_pd['Changed'] = 'N'

# Iterate through the rows of the pandas DataFrame containing the mappings
for _, row in df_mappings.iterrows():
    original_name = row['Original_Name']
    city = row['City']
    new_name = row['New_Name']
    new_city = row['New_City']

    # Create a mask for rows where replacements should happen
    mask = (df_pd["Name"] == original_name) & (df_pd["City"] == city)

    # Initialize an empty string to store changes for each row
    changes = []

    # Check if any row actually needs a replacement for the 'Name'
    if mask.any() and df_pd.loc[mask, "Name"].any():
        df_pd.loc[mask, "Name"] = new_name
        changes.append('Name')

    # Check if any row actually needs a replacement for the 'City'
    if mask.any() and df_pd.loc[mask, "City"].any():
        df_pd.loc[mask, "City"] = new_city
        changes.append('City')

    # Update the 'Changed' column with the fields that were changed, join them with ', '
    if changes:
        df_pd.loc[mask, 'Changed'] = ', '.join(changes)
    else:
        df_pd.loc[mask, 'Changed'] = 'N'

# Convert the updated pandas DataFrame back to a Polars DataFrame
df_updated = pl.from_pandas(df_pd)

print(df_updated)

shape: (4, 4)
┌─────────┬──────────┬─────┬────────────┐
│ Name    ┆ City     ┆ Age ┆ Changed    │
│ ---     ┆ ---      ┆ --- ┆ ---        │
│ str     ┆ str      ┆ i64 ┆ str        │
╞═════════╪══════════╪═════╪════════════╡
│ Alice   ┆ New York ┆ 25  ┆ N          │
│ Robert  ┆ Lublin   ┆ 30  ┆ Name, City │
│ Charlie ┆ Chicago  ┆ 35  ┆ N          │
│ Bob     ┆ New York ┆ 40  ┆ N          │
└─────────┴──────────┴─────┴────────────┘
